In [3]:
! pip install requests pandas gspread

In [4]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from data_project.tapdb import TapDBClient
from data_project.gsheets import DateSheet
from config import TAPDB_USER, TAPDB_PASS
import asyncio

target_date = (date.today() - timedelta(days=1)).isoformat()
#target_date = '2024-07-27'

if isinstance(target_date, str):
    target_date = date.fromisoformat(target_date)

async def create_tapdb():
    return TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

async def create_sheet():
    return DateSheet('[RP_EN] Revenue Report', date=target_date.isoformat())

task_tapdb = asyncio.create_task(create_tapdb())
task_sheet = asyncio.create_task(create_sheet())

await task_tapdb
await task_sheet

tapdb = task_tapdb.result()
sheet = task_sheet.result()

In [5]:
# [RP_EN] Revenue Report >> TapDB >> Android
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'pay', dim='os', device=True,
                flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

col_to_col = [
    ('收入', 'income', int),
    ('付費人數','paidPlayers', int),
    ('付費次數','payTimes', int),
    ('ARPU','ARPU', float),
    ('ARPPU','ARPPU', float),
    ('付費率','payRate', lambda x: float(x / 100))]
for col_sheet, col_data, func in col_to_col:
    sheet.update_cells(col_sheet, func(data[col_data].sum()))

data

,key,income,paidPlayers,payTimes,refund,refundUsers,refundTimes,ARPU,ARPPU,payRate
key,,,,,,,,,,
Android,Android,16556,538,1258,0,0,0,1.18,30.77,3.83


In [6]:
# [RP_EN] Revenue Report >> TapDB >> Android
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'active', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

sheet.update_cells('活躍裝置', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新增裝置', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,14752,2392,12360,11714,10403,9045,7131


In [7]:
# [RP_EN] Revenue Report >> TapDB >> n2i
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'pay', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

col_to_col = [
    ('收入n2i', 'income', int),
    ('付費人數n2i','paidPlayers', int),
    ('付費次數n2i','payTimes', int),
    ('ARPUn2i','ARPU', float),
    ('ARPPUn2i','ARPPU', float),
    ('付費率n2i','payRate', lambda x: float(x / 100))]
for col_sheet, col_data, func in col_to_col:
    sheet.update_cells(col_sheet, func(data[col_data].sum()))

data

,key,income,paidPlayers,payTimes,refund,refundUsers,refundTimes,ARPU,ARPPU,payRate
key,,,,,,,,,,
Android,Android,3689,80,223,0,0,0,1.86,46.11,4.03


In [8]:
# [RP_EN] Revenue Report >> TapDB >> n2i
sheet.change_sheet('TapDB', headers_row=2)
data = tapdb.get_data('RP NTK (PRO)', 'active', dim='os', device=True,
                      flts=[{'k': "dev", 'l': "=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])

sheet.update_cells('活躍裝置n2i', int(data['DAU_DEVICE'].sum()))
sheet.update_cells('新增裝置n2i', int(data['newDAU_DEVICE'].sum()))
data

,key,DAU_DEVICE,newDAU_DEVICE,DAU_DEVICE2,DAU_DEVICE3,DAU_DEVICE7,DAU_DEVICE14,DAU_DEVICE30
key,,,,,,,,
Android,Android,5711,5709,2,0,0,0,0


In [9]:
#[RP_EN] Revenue Report >> 設備留存
data = tapdb.get_data('RP NTK (PRO)', 'retain', begin_date='2024-03-04', pf='and', type='both',
                      flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])
data.sort_index(inplace=True)
data['key'] = data['key'].astype(str)
data_drr = data.loc[:, data.columns.str.contains('DRR')]
data = pd.concat([data.loc[:, ['newDevices']], data_drr / 100], axis=1)

data.to_csv(r'D:\Serena_Yu\Desktop\設備留存\retain.csv')
data

,newDevices,DRR1,DRR2,DRR3,DRR4,DRR5,DRR6,DRR7,DRR8,DRR9,...,DRR26,DRR27,DRR28,DRR29,DRR30,DRR60,DRR90,DRR120,DRR150,DRR180
key,,,,,,,,,,,,,,,,,,,,,
2024-03-03,80,0.4625,0.3750,0.3375,0.3250,0.3250,0.2625,0.3125,0.2500,0.2500,...,0.1500,0.1500,0.1625,0.1500,0.1125,0.0625,0.0500,0.0250,NaN,NaN
2024-03-04,7333,0.6464,0.5365,0.4987,0.4716,0.4519,0.4444,0.4259,0.3932,0.3803,...,0.2256,0.2187,0.2249,0.2112,0.2040,0.0997,0.0720,0.0520,NaN,NaN
2024-03-05,9156,0.4811,0.4040,0.3727,0.3461,0.3336,0.3177,0.2994,0.2866,0.2704,...,0.1547,0.1556,0.1513,0.1430,0.1418,0.0667,0.0470,0.0316,NaN,NaN
2024-03-06,7965,0.4359,0.3406,0.3101,0.2884,0.2719,0.2573,0.2467,0.2388,0.2213,...,0.1248,0.1163,0.1165,0.1105,0.1105,0.0551,0.0345,0.0231,NaN,NaN
2024-03-07,7855,0.3990,0.3120,0.2768,0.2545,0.2354,0.2358,0.2199,0.2042,0.1947,...,0.1094,0.1034,0.1018,0.0965,0.0923,0.0447,0.0281,0.0206,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25,2288,0.2872,0.1726,0.1416,0.1342,0.0044,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-26,2414,0.2788,0.1810,0.1479,0.0120,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-27,2143,0.2921,0.1853,0.0098,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
target_date = (date.today() - timedelta(days=1)).isoformat()
#target_date = '2024-07-28'

tapdb = TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date, tz_offset=-12)

target_date = tapdb.target_date.isoformat()
sheet = DateSheet('[RP_EN] Revenue Report', date=target_date)

In [15]:
# [RP_EN] Revenue Report >> 轉化率
data = tapdb.get_data('RP NTK (PRO)', 'source', dim='ver',
                      flts=[{'k': "dev", 'l': "!=", 'v': ["n2i mono arm arm64 n2i", "n2i multi arm arm64 n2i"]}])
row = data.loc[max(data.index)]

sheet.change_sheet('轉化率', headers_row=2)
sheet.update_cells('版本', row['key'])
sheet.update_cells('新增設備', row['newDevices'])
sheet.update_cells('轉化設備', row['convertedDevices'])

data

,key,newPlayers,totalCharge,totalChargeAmount,firstChargePlayers,firstChargeAmount,newDevices,convertedDevices,devicesConvertRate,dayRetainedRate1,dayRetainedRate7,chargeRate,firstChargeRate,ARPU,ARPPU
key,,,,,,,,,,,,,,,
8.0.0.170,8.0.0.170,2234,23,446,23,438,2258,2103,93.14,None,None,1.02,1.02,0.19,19.04
1.0.0.139,1.0.0.139,0,0,0,0,0,122,14,11.48,None,None,0.00,0.00,0.00,0.00
1.0.0.160,1.0.0.160,0,0,0,0,0,10,2,20.00,None,None,0.00,0.00,0.00,0.00
1.0.0.156,1.0.0.156,0,0,0,0,0,1,0,0.00,None,None,0.00,0.00,0.00,0.00
1.0.0.140,1.0.0.140,0,0,0,0,0,1,0,0.00,None,None,0.00,0.00,0.00,0.00


In [12]:
target_date = (date.today() - timedelta(days=1)).isoformat()

tapdb = TapDBClient(TAPDB_USER, TAPDB_PASS, target_date=target_date)

target_date = tapdb.target_date.isoformat()
sheet = DateSheet('[RP_EN] Revenue Report', date=target_date)

In [13]:
# [RP_EN] Revenue Report >> Wkly Re.
sheet.change_sheet('Wkly Re.')
data = tapdb.get_data('RP NTK (PRO)', 'wau', device=True)
target_week=target_date-timedelta(days=target_date.weekday())
row=data.loc[target_week]

sheet.update_cells('活躍設備\n(tap db)', int(row['WAU_DEVICE']))


data = tapdb.get_data('RP NTK (PRO)', 'wau', device=True, pf='and')
target_week=target_date-timedelta(days=target_date.weekday())
row=data.loc[target_week]
sheet.update_cells('活躍設備\n(Android)', int(row['WAU_DEVICE']))

data

ValueError: Given date 2024-07-29 not found, min 日期 max 2024-08-18